In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [2]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [3]:
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [7]:
columns_sorted = files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True) [0]
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [13]:
def count_move_small_tables(table):
    df = pd.read_json(os.path.join(input+table), compression='gzip', lines=True)
    if len(df)<=10:
        df.to_json(os.path.join(output+'/under_10_rows/'+table), compression = 'gzip', lines = True)
        print(len(df))
        return
    else:
        row_list.extend([table,len(df)])
        print(row_list)
   
       

In [16]:
row_list = list()
for index in columns_sorted.filename:
    print(index)
    setcol = set(columns_sorted.loc[index]['listofcolumns'])
    print(setcol)
    input = '../../src/data/data/'+index+'/All/'
    output = '../../src/data/data/'+index
    if not os.path.exists(output):
        os.makedirs(output)
    #for _ in tqdm(count_move_small_tables, [file for file in os.columns_sorted(input) if file.endswith('.json.gz')]), total=len([file for file in os.columns_sortedinput) if file.endswith('.json.gz')]):
     #   pass
    count_move_small_tables
    #print(len([file for file in os.columns_sorted(output) if file.endswith('.json.gz')]))

Book
{'isbn', 'inlanguage', 'name', 'numberofpages', 'bookformat', 'bookedition', 'genre', 'datepublished', 'aggregaterating', 'publisher', 'offers', 'author'}
CreativeWork
{'inlanguage', 'datemodified', 'datecreated', 'datepublished', 'interactioncount', 'genre', 'articlebody', 'creator', 'copyrightyear', 'interactionstatistic', 'version', 'interactiontype', 'mainentityofpage', 'copyrightholder', 'name', 'commentcount', 'aggregaterating', 'keywords', 'author', 'headline', 'publisher', 'offers', 'alternativeheadline'}
Event
{'name', 'performers', 'enddate', 'eventstatus', 'duration', 'eventattendancemode', 'location', 'doortime', 'organizer', 'performer', 'typicalagerange'}
Hotel
{'pricerange', 'name', 'address', 'starrating', 'aggregaterating', 'geo', 'email', 'telephone'}
LocalBusiness
{'contactpoint', 'streetaddress', 'geo', 'email', 'areaserved', 'legalname', 'addressregion', 'owns', 'founder', 'citystatezip', 'sameas', 'telephone', 'addresslocality', 'name', 'address', 'hasmap', '